In [1]:
import pandas as pd

In [2]:
room_list_detail = pd.read_csv('C:/Users/daun5/jupyter_notebook/room_list_공릉동.csv', encoding='cp949')

In [71]:
train_data = [{'room_id' : 'room_id', 
               'room_size' : 'room_size', 
               'Deposit' : 'Deposit', 
               'totalcost_month' : 'totalcost_month'}]
trash_data = [{'[' : '[',
              ']' : ']',
              ',' : ','}]

In [5]:
i = 0;
for data in room_list_detail['maintenance_cost_str']:
    if data == '없음':
        room_list_detail['maintenance_cost_str'][i] = 0
    else:
        room_list_detail['maintenance_cost_str'][i] = float(data[:-3])
    i = i + 1;
    


<ipython-input-5-508760a48303>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_list_detail['maintenance_cost_str'][i] = float(data[:-3])
<ipython-input-5-508760a48303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_list_detail['maintenance_cost_str'][i] = 0


In [72]:
for data in room_list_detail['price_info_str']:
    j = 0
    trash_data = []
    for i in data:
        if i != '[' and i != ']' and i != """'""":
            train_data.append(i)
        else:
            trash_data.append(i)
train_data

[{'room_id': 'room_id',
  'room_size': 'room_size',
  'Deposit': 'Deposit',
  'totalcost_month': 'totalcost_month'},
 '3',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '3',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '9',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '4',
 '0',
 '5',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '8',
 '1',
 '0',
 '0',
 '0',
 ',',
 ' ',
 '4',
 '5',
 '3',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '8',
 '3',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '8',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '0',
 '2',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '5',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '0',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '2',
 '1',
 '0',
 '0',
 '0',
 ',',
 ' ',
 '4',
 '1',
 '5',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '3',
 '5',
 '0',
 ',',
 ' ',
 '3',
 '9',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '9',
 '1',
 '억',
 '3',
 '0',
 '0',
 '0',
 ',',
 ' ',
 '4',
 '7',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '3',
 '3',
 '0',
 '0',
 ',',
 ' ',
 '2',
 '8',
 '3',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '4',
 '1',
 '0',
 '0',
 ',',
 ' ',
 '3',
 '0',
 '3',
 '0',
 '0',
 

In [ ]:
with open('C:/Users/daun5/jupyter_notebook/train_data_공릉동.csv', mode = 'w', newline='') as train_data:
    train_data_writer = csv.writer(train_data)

    for data in train_data:
        train_data_writer.writerow([data['room_id'], data['room_size'], data['totalcost_month']

In [ ]:
train_data.append(
             {
                 'room_id' : room_list_detail['room_id'],
                 'room_size' : room_list_detail['room_size'],
                 'maintenance_cost_str' : room_list_detail['maintenance_cost_str'],
                 'price_info_str' : room_list_detail['price_info_str']
             }
        )
 if data[i] != '[' and data[i] != "'":
            train_data['Deposit'][i] = data[i]

In [28]:
room_list_detail['price_info_str'][0]

"[['300', '33']]"

In [45]:
numbers = ['one', 'two', 'three', 'four', 'five']
    
for n in numbers:
    print(n)

one
two
three
four
five


In [51]:
train_data['Deposit']

TypeError: list indices must be integers or slices, not str

In [50]:
room_list_detail['price_info_str']

0       [['300', '33']]
1       [['100', '39']]
2       [['100', '40']]
3       [['500', '38']]
4      [['1000', '45']]
             ...       
714    [['1000', '38']]
715     [['500', '48']]
716    [['3000', '30']]
717     [['300', '30']]
718     [['500', '55']]
Name: price_info_str, Length: 719, dtype: object